# സെഷൻ 4 – SLM vs LLM താരതമ്യം

Foundry Local വഴി പ്രവർത്തിക്കുന്ന ഒരു ചെറിയ ഭാഷാ മോഡലും വലിയ മോഡലും തമ്മിലുള്ള ലാറ്റൻസിയും സാമ്പിൾ പ്രതികരണ ഗുണനിലവാരവും താരതമ്യം ചെയ്യുക.


## ⚡ ക്വിക്ക് സ്റ്റാർട്ട്

**മെമ്മറി-ഓപ്റ്റിമൈസ്ഡ് സെറ്റപ്പ് (അപ്ഡേറ്റ് ചെയ്തത്):**
1. മോഡലുകൾ സ്വയം CPU വകഭേദങ്ങൾ തിരഞ്ഞെടുക്കുന്നു (ഏതെങ്കിലും ഹാർഡ്‌വെയറിൽ പ്രവർത്തിക്കും)
2. 7B പകരം `qwen2.5-3b` ഉപയോഗിക്കുന്നു (~4GB RAM ലാഭിക്കുന്നു)
3. പോർട്ട് സ്വയം കണ്ടെത്തൽ (മാനുവൽ കോൺഫിഗറേഷൻ ആവശ്യമില്ല)
4. മൊത്തം ആവശ്യമായ RAM: ~8GB ശുപാർശ ചെയ്യുന്നു (മോഡലുകളും OS ഉം)

**ടെർമിനൽ സെറ്റപ്പ് (30 സെക്കൻഡ്):**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

അപ്പോൾ ഈ നോട്ട്‌ബുക്ക് റൺ ചെയ്യൂ! 🚀


### വിശദീകരണം: ആശ്രിത ഇൻസ്റ്റലേഷൻ
ടൈമിംഗ് மற்றும் ചാറ്റ് അഭ്യർത്ഥനകൾക്കായി ആവശ്യമായ കുറഞ്ഞ പാക്കേജുകൾ (`foundry-local-sdk`, `openai`, `numpy`) ഇൻസ്റ്റാൾ ചെയ്യുന്നു. പുനരാവർത്തനശീലമായി സുരക്ഷിതമായി വീണ്ടും പ്രവർത്തിപ്പിക്കാം.


# സീനാരിയോ
ഒരു പ്രതിനിധി ചെറിയ ഭാഷാ മോഡലിനെ (SLM) ഒരു വലിയ മോഡലുമായി ഒരു പ്രോംപ്റ്റിൽ താരതമ്യം ചെയ്ത് ട്രേഡ്‑ഓഫുകൾ കാണിക്കുക:
- **ലേറ്റൻസി വ്യത്യാസം** (വാൾ ക്ലോക്ക് സെക്കൻഡുകൾ)
- **ടോക്കൺ ഉപയോഗം** (ലഭ്യമായെങ്കിൽ) ത്രൂപുട്ടിന്റെ പ്രതീകമായി
- **ഉദാഹരണ ഗുണനിലവാര ഔട്ട്‌പുട്ട്** വേഗത്തിൽ പരിശോധിക്കാൻ
- **സ്പീഡപ്പ് കണക്കുകൂട്ടൽ** പ്രകടന നേട്ടങ്ങൾ അളക്കാൻ

**പരിസ്ഥിതി വ്യത്യാസങ്ങൾ:**
- `SLM_ALIAS` - ചെറിയ ഭാഷാ മോഡൽ (ഡീഫോൾട്ട്: phi-4-mini, ~4GB RAM)
- `LLM_ALIAS` - വലിയ ഭാഷാ മോഡൽ (ഡീഫോൾട്ട്: qwen2.5-7b, ~7GB RAM)
- `COMPARE_PROMPT` - താരതമ്യത്തിനുള്ള ടെസ്റ്റ് പ്രോംപ്റ്റ്
- `COMPARE_RETRIES` - പുനരായോഗ ശ്രമങ്ങൾ (ഡീഫോൾട്ട്: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - സർവീസ് എൻഡ്‌പോയിന്റ് ഓവർറൈഡ് ചെയ്യുക (സജ്ജമല്ലെങ്കിൽ സ്വയം കണ്ടെത്തും)

**എങ്ങനെ പ്രവർത്തിക്കുന്നു (അധികൃത SDK മാതൃക):**
1. **FoundryLocalManager** ഫൗണ്ടറി ലോക്കൽ സർവീസ് ആരംഭിക്കുകയും നിയന്ത്രിക്കുകയും ചെയ്യുന്നു
2. സർവീസ് പ്രവർത്തനരഹിതമായാൽ സ്വയം ആരംഭിക്കുന്നു (മാനുവൽ ക്രമീകരണം ആവശ്യമില്ല)
3. മോഡലുകൾ അലിയാസുകളിൽ നിന്ന് കൺക്രീറ്റ് ഐഡികളിലേക്ക് സ്വയം പരിഹരിക്കുന്നു
4. ഹാർഡ്‌വെയർ-ഓപ്റ്റിമൈസ്ഡ് വകഭേദങ്ങൾ തിരഞ്ഞെടുക്കുന്നു (CUDA, NPU, അല്ലെങ്കിൽ CPU)
5. OpenAI-സമാന ക്ലയന്റ് ചാറ്റ് പൂർത്തീകരണങ്ങൾ നടത്തുന്നു
6. മെട്രിക്‌സ് പിടിക്കുന്നു: ലേറ്റൻസി, ടോക്കൺസ്, ഔട്ട്‌പുട്ട് ഗുണനിലവാരം
7. ഫലങ്ങൾ താരതമ്യം ചെയ്ത് സ്പീഡപ്പ് അനുപാതം കണക്കാക്കുന്നു

ഈ മൈക്രോ-താരതമ്യം നിങ്ങളുടെ ഉപയോഗത്തിനായി വലിയ മോഡലിലേക്ക് റൂട്ടിംഗ് ന്യായീകരിക്കേണ്ട സമയത്തെ തീരുമാനിക്കാൻ സഹായിക്കുന്നു.

**SDK റഫറൻസ്:** 
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- വർക്ക്‌ഷോപ്പ് ഉപകരണങ്ങൾ: ../samples/workshop_utils.py-ൽ നിന്നുള്ള അധികൃത മാതൃക ഉപയോഗിക്കുന്നു

**പ്രധാന ഗുണങ്ങൾ:**
- ✅ സ്വയം സർവീസ് കണ്ടെത്തലും ആരംഭവും
- ✅ പ്രവർത്തനരഹിതമായാൽ സ്വയം സർവീസ് ആരംഭിക്കൽ
- ✅ ഇൻബിൽറ്റ് മോഡൽ പരിഹാരവും കാഷിംഗും
- ✅ ഹാർഡ്‌വെയർ ഓപ്റ്റിമൈസേഷൻ (CUDA/NPU/CPU)
- ✅ OpenAI SDK അനുയോജ്യത
- ✅ പുനരായോഗങ്ങളോടെയുള്ള ശക്തമായ പിശക് കൈകാര്യം
- ✅ ലോക്കൽ ഇൻഫറൻസ് (ക്ലൗഡ് API ആവശ്യമില്ല)


## 🚨 മുൻ‌വൈദ്യങ്ങൾ: Foundry Local പ്രവർത്തനക്ഷമമായിരിക്കണം!

**ഈ നോട്ട്‌ബുക്ക് പ്രവർത്തിപ്പിക്കുന്നതിന് മുമ്പ്**, Foundry Local സേവനം സജ്ജമാക്കിയിട്ടുണ്ടെന്ന് ഉറപ്പാക്കുക:

### ക്വിക്ക് സ്റ്റാർട്ട് കമാൻഡുകൾ (ടെർമിനലിൽ പ്രവർത്തിപ്പിക്കുക):

```bash
# 1. ഫൗണ്ട്രി ലോക്കൽ സർവീസ് ആരംഭിക്കുക
foundry service start

# 2. ഈ താരതമ്യത്തിൽ ഉപയോഗിക്കുന്ന ഡിഫോൾട്ട് മോഡലുകൾ ലോഡ് ചെയ്യുക (CPU-ഓപ്റ്റിമൈസ്ഡ്)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. മോഡലുകൾ ലോഡ് ചെയ്തിട്ടുണ്ടെന്ന് സ്ഥിരീകരിക്കുക
foundry model ls

# 4. സർവീസ് ആരോഗ്യ സ്ഥിതി പരിശോധിക്കുക
foundry service status
```

### ബദൽ മോഡലുകൾ (ഡീഫോൾട്ടുകൾ ലഭ്യമല്ലെങ്കിൽ):

```bash
# വളരെ കുറവ് മെമ്മറി ഉണ്ടെങ്കിൽ പോലും ചെറുതായുള്ള മറ്റ് വഴികൾ
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# അല്ലെങ്കിൽ ഈ നോട്ട്‌ബുക്കിൽ പരിസ്ഥിതി വ്യത്യാസങ്ങൾ അപ്ഡേറ്റ് ചെയ്യുക:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # അല്ലെങ്കിൽ കുറഞ്ഞ മെമ്മറിയ്ക്കായി qwen2.5-0.5b
```

⚠️ **ഈ ഘട്ടങ്ങൾ ഒഴിവാക്കുകയാണെങ്കിൽ**, താഴെ നോട്ട്‌ബുക്ക് സെല്ലുകൾ പ്രവർത്തിപ്പിക്കുമ്പോൾ `APIConnectionError` കാണും.


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### Explanation: Core Imports
സമയം സംബന്ധിച്ച ഉപകരണങ്ങളും മോഡൽ വിവരങ്ങൾ നേടാനും ചാറ്റ് പൂർത്തീകരണങ്ങൾ നടത്താനും ഉപയോഗിക്കുന്ന Foundry Local / OpenAI ക്ലയന്റുകളും കൊണ്ടുവരുന്നു.


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### വിശദീകരണം: ഉപനാമങ്ങളും പ്രോംപ്റ്റ് ക്രമീകരണവും
ചെറിയ മോഡലിനും വലിയ മോഡലിനും വേണ്ടി പരിസ്ഥിതി-ക്രമീകരിക്കാവുന്ന ഉപനാമങ്ങൾ നിർവചിക്കുന്നു കൂടാതെ ഒരു താരതമ്യ പ്രോംപ്റ്റും. വ്യത്യസ്ത മോഡൽ കുടുംബങ്ങളോ ജോലികളോ പരീക്ഷിക്കാൻ പരിസ്ഥിതി വ്യത്യാസങ്ങൾ ക്രമീകരിക്കുക.


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 മെമ്മറി-ഓപ്റ്റിമൈസ്ഡ് കോൺഫിഗറേഷൻ

**ഈ നോട്ട്‌ബുക്ക് ഡീഫോൾട്ടായി മെമ്മറി-ക്ഷമമായ മോഡലുകൾ ഉപയോഗിക്കുന്നു:**
- `phi-4-mini` → ~4GB RAM (Foundry Local CPU വേരിയന്റ് സ്വയം തിരഞ്ഞെടുക്കുന്നു)
- `qwen2.5-3b` → ~3GB RAM (7B മോഡലിന് പകരം, അത് ~7GB+ ആവശ്യമാണ്)

**പോർട്ട് സ്വയം കണ്ടെത്തൽ:**
- Foundry Local വ്യത്യസ്ത പോർട്ടുകൾ ഉപയോഗിക്കാം (സാധാരണയായി 55769 അല്ലെങ്കിൽ 59959)
- താഴെയുള്ള ഡയഗ്നോസ്റ്റിക് സെൽ ശരിയായ പോർട്ട് സ്വയം കണ്ടെത്തും
- മാനുവൽ കോൺഫിഗറേഷൻ ആവശ്യമില്ല!

**നിങ്ങൾക്ക് കുറവ് RAM (<8GB) ഉണ്ടെങ്കിൽ, കൂടുതൽ ചെറിയ മോഡലുകൾ ഉപയോഗിക്കുക:**
```python
SLM = 'phi-3.5-mini'      # ~2ജിബി
LLM = 'qwen2.5-0.5b'      # ~500എംബി
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### وضاحت: تنفيذ المساعدين (نموذج Foundry SDK)
يستخدم نموذج Foundry Local SDK الرسمي كما هو موثق في عينات ورشة العمل:

**النهج:**
- **FoundryLocalManager** - يقوم بتهيئة وإدارة خدمة Foundry Local
- **الكشف التلقائي** - يكتشف نقطة النهاية تلقائيًا ويتعامل مع دورة حياة الخدمة
- **حل النموذج** - يحل الأسماء المستعارة إلى معرفات النموذج الكاملة (مثل phi-4-mini → phi-4-mini-instruct-cpu)
- **تحسين الأجهزة** - يختار أفضل نسخة للأجهزة المتاحة (CUDA، NPU، أو CPU)
- **عميل OpenAI** - مهيأ بنقطة نهاية المدير للوصول إلى واجهة برمجة تطبيقات متوافقة مع OpenAI

**ميزات المرونة:**
- منطق إعادة المحاولة بتراجع أسي (قابل للتكوين عبر البيئة)
- بدء الخدمة تلقائيًا إذا لم تكن قيد التشغيل
- التحقق من الاتصال بعد التهيئة
- التعامل مع الأخطاء بسلاسة مع تقارير أخطاء مفصلة
- تخزين النموذج مؤقتًا لتجنب التهيئة المتكررة

**هيكل النتيجة:**
- قياس الكمون (وقت الساعة الحائطية)
- تتبع استخدام الرموز (إذا كان متاحًا)
- عينة من الإخراج (مقتطعة للقراءة)
- تفاصيل الأخطاء للطلبات الفاشلة

يستفيد هذا النموذج من وحدة workshop_utils التي تتبع نموذج SDK الرسمي.

**مرجع SDK:**
- المستودع الرئيسي: https://github.com/microsoft/Foundry-Local
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### വിശദീകരണം: പ്രീ-ഫ്ലൈറ്റ് സെൽഫ്-ടെസ്റ്റ്
രണ്ടു മോഡലുകൾക്കും FoundryLocalManager ഉപയോഗിച്ച് ഒരു ലഘു കണക്റ്റിവിറ്റി പരിശോധന നടത്തുന്നു. ഇത് സ്ഥിരീകരിക്കുന്നു:
- സേവനം ആക്സസിബിളാണ്
- മോഡലുകൾ ആരംഭിക്കാനാകും
- അലിയാസുകൾ യഥാർത്ഥ മോഡൽ ഐഡികളിലേക്ക് പരിഹരിക്കുന്നു
- താരതമ്യം നടത്തുന്നതിന് മുമ്പ് കണക്ഷൻ സ്ഥിരമാണ്

setup() ഫംഗ്ഷൻ workshop_utils-ൽ നിന്നുള്ള ഔദ്യോഗിക SDK പാറ്റേൺ ഉപയോഗിക്കുന്നു.


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ വേഗത്തിലുള്ള പരിഹാരം: നോട്ട്‌ബുക്കിൽ നിന്ന് ഫൗണ്ട്രി ലോക്കൽ ആരംഭിക്കുക (ഐച്ഛികം)

മുകളിൽ കാണിച്ച ഡയഗ്നോസ്റ്റിക് സർവീസ് പ്രവർത്തിക്കുന്നില്ലെന്ന് കാണിച്ചാൽ, നിങ്ങൾക്ക് ഇവിടെ നിന്ന് അത് ആരംഭിക്കാൻ ശ്രമിക്കാം:

**കുറിപ്പ്:** ഇത് വിൻഡോസിൽ ഏറ്റവും നല്ല രീതിയിൽ പ്രവർത്തിക്കുന്നു. മറ്റ് പ്ലാറ്റ്‌ഫോമുകളിൽ, ടെർമിനൽ കമാൻഡുകൾ ഉപയോഗിക്കുക.


### ⚠️ കണക്ഷൻ പിശകുകൾ പരിഹരിക്കൽ

നിങ്ങൾക്ക് `APIConnectionError` കാണിച്ചാൽ, Foundry Local സേവനം പ്രവർത്തിക്കുന്നില്ല അല്ലെങ്കിൽ മോഡലുകൾ ലോഡ് ചെയ്തിട്ടില്ല. ഈ ഘട്ടങ്ങൾ പരീക്ഷിക്കുക:

**1. സേവനത്തിന്റെ നില പരിശോധിക്കുക:**
```bash
# ഒരു ടെർമിനലിൽ (നോട്ട്ബുക്കിൽ അല്ല):
foundry service status
```

**2. സേവനം ആരംഭിക്കുക (പ്രവർത്തിക്കുന്നില്ലെങ്കിൽ):**
```bash
foundry service start
```

**3. ആവശ്യമായ മോഡലുകൾ ലോഡ് ചെയ്യുക:**
```bash
# താരതമ്യത്തിനായി ആവശ്യമായ മോഡലുകൾ ലോഡ് ചെയ്യുക
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# അല്ലെങ്കിൽ പകരം മോഡലുകൾ ഉപയോഗിക്കുക:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. മോഡലുകൾ ലഭ്യമാണ് എന്ന് സ്ഥിരീകരിക്കുക:**
```bash
foundry model ls
```

**സാധാരണ പ്രശ്നങ്ങൾ:**
- ❌ സേവനം പ്രവർത്തിക്കുന്നില്ല → `foundry service start` പ്രവർത്തിപ്പിക്കുക
- ❌ മോഡലുകൾ ലോഡ് ചെയ്തിട്ടില്ല → `foundry model run <model-name>` പ്രവർത്തിപ്പിക്കുക
- ❌ പോർട്ട് സംഘർഷങ്ങൾ → മറ്റൊരു സേവനം പോർട്ട് ഉപയോഗിക്കുന്നുണ്ടോ എന്ന് പരിശോധിക്കുക
- ❌ ഫയർവാൾ തടയുന്നു → പ്രാദേശിക കണക്ഷനുകൾ അനുവദിച്ചിട്ടുണ്ടെന്ന് ഉറപ്പാക്കുക

**വേഗത്തിലുള്ള പരിഹാരം:** പ്രീ-ഫ്ലൈറ്റ് പരിശോധനയ്ക്ക് മുമ്പ് താഴെയുള്ള ഡയഗ്നോസ്റ്റിക് സെൽ പ്രവർത്തിപ്പിക്കുക.


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ പ്രീ-ഫ്ലൈറ്റ് ചെക്ക്: മോഡൽ ലഭ്യത

ഈ സെൽ താരതമ്യം നടത്തുന്നതിന് മുമ്പ് കോൺഫിഗർ ചെയ്ത എൻഡ്‌പോയിന്റിൽ രണ്ട് മോഡലുകളും എത്തിച്ചേരാൻ കഴിയുന്നുണ്ടോ എന്ന് പരിശോധിക്കുന്നു.


### Explanation: Run Comparison & Collect Results
ഔദ്യോഗിക Foundry SDK പാറ്റേൺ ഉപയോഗിച്ച് രണ്ട് അലിയാസുകളിലൂടെയും ഇറ്ററേറ്റ് ചെയ്യുന്നു:
1. ഓരോ മോഡലും setup() ഉപയോഗിച്ച് ആരംഭിക്കുക (FoundryLocalManager ഉപയോഗിക്കുന്നു)
2. OpenAI-സമാനമായ API ഉപയോഗിച്ച് ഇൻഫറൻസ് നടത്തുക
3. ലാറ്റൻസി, ടോക്കൺസ്, സാമ്പിൾ ഔട്ട്പുട്ട് പിടിക്കുക
4. താരതമ്യ വിശകലനത്തോടെ JSON സംഗ്രഹം സൃഷ്ടിക്കുക

ഇത് session04/model_compare.py-യിലെ വർക്‌ഷോപ്പ് സാമ്പിളുകളിലെ സമാന പാറ്റേൺ പിന്തുടരുന്നു.


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### ഫലങ്ങൾ വ്യാഖ്യാനിക്കൽ

**പ്രധാന മെട്രിക്‌സ്:**
- **ലേറ്റൻസി**: കുറവായിരിക്കുക നല്ലത് - വേഗത്തിലുള്ള പ്രതികരണ സമയം സൂചിപ്പിക്കുന്നു
- **ടോക്കൺസ്**: ഉയർന്ന ത്രൂപുട്ട് = കൂടുതൽ ടോക്കൺസ് പ്രോസസ്സ് ചെയ്യുന്നു
- **റൂട്ടു**: ഏത് API എൻഡ്‌പോയിന്റ് ഉപയോഗിച്ചതാണെന്ന് സ്ഥിരീകരിക്കുന്നു

**SLM vs LLM എപ്പോൾ ഉപയോഗിക്കണം:**
- **SLM (സ്മോൾ ലാംഗ്വേജ് മോഡൽ)**: വേഗത്തിലുള്ള പ്രതികരണങ്ങൾ, കുറവ് റിസോഴ്‌സ് ഉപയോഗം, ലളിതമായ ജോലികൾക്ക് നല്ലത്
- **LLM (ലാർജ് ലാംഗ്വേജ് മോഡൽ)**: ഉയർന്ന ഗുണമേന്മ, മികച്ച തർക്കനിർണയം, ഗുണമേന്മ പ്രധാനമായപ്പോൾ ഉപയോഗിക്കുക

**അടുത്ത ഘട്ടങ്ങൾ:**
1. വ്യത്യസ്ത പ്രോംപ്റ്റുകൾ പരീക്ഷിച്ച് സങ്കീർണ്ണത താരതമ്യത്തെ എങ്ങനെ ബാധിക്കുന്നു എന്ന് കാണുക
2. മറ്റ് മോഡൽ ജോഡികൾ പരീക്ഷിക്കുക
3. ടാസ്ക് സങ്കീർണ്ണത അടിസ്ഥാനമാക്കി ബുദ്ധിമുട്ടോടെ റൂട്ടുചെയ്യാൻ Workshop റൂട്ടർ സാമ്പിളുകൾ (സെഷൻ 06) ഉപയോഗിക്കുക


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖയാണ് പ്രാമാണികമായ ഉറവിടം എന്ന് പരിഗണിക്കേണ്ടതാണ്. നിർണായക വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനത്തിന്റെ ഉപയോഗത്തിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
